In [7]:
import os
from dotenv import load_dotenv
from groq import Groq
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [2]:
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

In [3]:
def get_completion(prompt, model="llama3-8b-8192"):
    client = Groq(api_key=groq_api_key)
    chat_completion = client.chat.completions.create(
                        messages = [{
                            "role":"user",
                            "content":prompt
                        }],
        model = model
    )
    return chat_completion.choices[0].message.content

In [4]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
style = """American English \
in a calm and respectful tone
"""
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

In [5]:
print(get_completion(prompt, 'mixtral-8x7b-32768'))

I'm really upset that the lid of my blender flew off and splattered my kitchen walls with smoothie! To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I need your help right now, friend!


In [8]:
chat = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")
chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000238A6956B90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000238A71325D0>, model_name='llama3-8b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [9]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [10]:
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [11]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [12]:
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [13]:
customer_messages = prompt_template.format_messages(
                    style = customer_style,
                    text = customer_email)

In [14]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [15]:
customer_messages[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")

In [14]:
response = chat(customer_messages)
print(response.content)

D:\anaconda\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Here is the translated text in American English, written in a calm and respectful tone:

"I'm extremely frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I really need your help right now."

Here's the original text with the requested translation:

```
Original: Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
Translated: I'm extremely frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I really need your help right now.
```


In [16]:
system = '''
You are a highly skilled language translation model. Your task is to translate text from one language to another with accuracy and attention to context. Please adhere to the following guidelines:

1. **Accuracy**: Ensure that the translation is accurate and conveys the same meaning as the original text.
2. **Context Sensitivity**: Maintain the appropriate tone, style, and context of the original text, whether it is formal, informal, technical, or conversational.
3. **Idiomatic Expressions**: Translate idiomatic expressions in a way that preserves their meaning in the target language.
4. **Cultural Relevance**: Adapt cultural references to be relevant and understandable to the target audience.
5. **Clarity and Fluency**: Ensure that the translated text is clear, fluent, and grammatically correct in the target language.
6. **Consistency**: Maintain consistent terminology, especially for technical terms or specialized vocabulary.

Instructions:
1. **General Translation**: For standard translations, provide a direct and accurate translation while maintaining the tone and context.
2. **Context-Specific Translation**: If specified, adjust the translation to fit the given context (e.g., business, casual conversation, technical manual).
3. **Formal to Informal Translation**: When required, adjust the tone from formal to informal, ensuring it is suitable for a casual conversation.
4. **Localization**: Adapt the translation for localization purposes, making it culturally relevant and understandable to the target audience.

Example Inputs:
1. **General Translation**:
   - Source Language: English
   - Target Language: Spanish
   - Text: "Hello, how are you?"

2. **Context-Specific Translation**:
   - Source Language: English
   - Target Language: German
   - Text: "Please review the attached financial report and provide your feedback."

3. **Formal to Informal Translation**:
   - Source Language: English
   - Target Language: French
   - Text: "Could you please send me the document at your earliest convenience?"

4. **Localization**:
   - Source Language: English
   - Target Language: Japanese
   - Text: "We're planning a Fourth of July barbecue this weekend."

Your role is to provide translations that adhere to these guidelines. Let's begin.

'''
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
response = chain.invoke({"text": customer_messages})
print(response.content)

Here is the translated text in American English, in a calm and respectful tone:

```
I'm extremely frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I really need your help right now.
```

Note: I've maintained the informal tone and language used in the original text, while adapting it to American English and a calm and respectful tone.


In [17]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [18]:
customer_messages = prompt_template.format_messages(
                    style = service_style_pirate,
                    text = service_reply)

In [19]:
customer_messages[0].content

"Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"

In [20]:
response = chain.invoke({'text': customer_messages})
print(response.content)

Arrr, shiver me timbers!

Here be the translated text, matey:

```Hey there, matey! Sorry to say, but our warranty don't cover the cost o' cleanin' up yer mess, savvy? Ye see, ye forgot to put the lid on yer blender, and that's why ye got yerself into this pickle. Tough luck, matey! May the winds o' fortune blow in yer favor next time. Fair winds and following seas! See ye later, me hearty!```

I hope this meets yer requirements, matey!


## Output Parser

In [21]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [22]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [23]:
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n')

In [24]:
prompt_template.messages[0].prompt.input_variables

['text']

In [25]:
customer_messages = prompt_template.format_messages(
                    text = customer_review)

In [25]:
customer_messages[0].content

"For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n\n"

In [26]:
system = '''
You are a helpful ai assistant
'''
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
response = chain.invoke({"text": customer_messages})
print(response.content)

Here is the extracted information in JSON format:

```
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"],
    "text": "This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [27]:
type(response.content)

str

## Parse the LLM output string into a Python dictionary

In [28]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [29]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [30]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [31]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')])

In [32]:
format_instructions = output_parser.get_format_instructions()

In [33]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [34]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [35]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [36]:
system = '''
You are a helpful ai assistant
'''
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
response = chain.invoke({"text": messages})
print(response.content)

Here is the extracted information in the required format:

```json
{
    "gift": "True", // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
    "delivery_days": "2", // How many days did it take for the product to arrive? If this information is not found, output -1.
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."] // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```
